In [1]:
cd(@__DIR__) #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using CSV
using RData
using Statistics
using Dates
using HypothesisTests

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`



R version 3.6.1 (2019-07-05) -- "Action of the Toes"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



┌ Warning: RCall.jl: [Previously saved workspace restored]
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113


In [2]:
R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)

SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest, 
                                                  StandardValues, 
                                                  reps, 
                                                  PSE_Difference, 
                                                  JND_Difference, 
                                                  Multiplicator_PSE_Standard, 
                                                  Multiplicator_SD_Standard, 
                                                  SD_ResponseFunction, 
                                                  Mean_Variability_Between = 0.1, 
                                                  SD_Variability_Between = 0.1){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
  
  Psychometric = Psychometric %>%
    group_by(ID) %>%#
    mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
           SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
  
  Psychometric = Psychometric %>%
    mutate(
      Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
      SD_Standard = StandardValues*Multiplicator_SD_Standard,
      Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
      SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
      staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
      Presented_TestStimulusStrength = Mean*staircase_factor,
      Difference = Presented_TestStimulusStrength - StandardValues,
      AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
      Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
    )
  
  Psychometric = Psychometric %>%
    filter(abs(staircase_factor-1) < 0.75) %>%
    group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
    mutate(Yes = sum(Answer==1),
           Total = length(ConditionOfInterest))
  
  Psychometric
}
""";

┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning: package 'tidyverse' was built under R version 3.6.3
│ -- Attaching packages --------------------------------------- tidyverse 1.3.0 --
│ v ggplot2 3.3.0     v purrr   0.3.3
│ v tibble  3.0.0     v stringr 1.4.0
│ v tidyr   1.0.2     v forcats 0.5.0
│ v readr   1.3.1     
│ Warning: package 'ggplot2' was built under R version 3.6.3
│ Warning: package 'tibble' was built under R version 3.6.3
│ Warning: package 'tidyr' was built under R version 3.6.3
│ Warning: package 'forcats' was built under R version 3.6.3
│ -- Conflicts ------------------------------------------ tidyverse_conflicts() --
│ x dplyr

In [3]:
function SimulateDataframe_Twolevel(n,
        ConditionOfInterest,
        StandardValues,
        reps,
        PSE_Difference,
        JND_Difference,
        Multiplicator_PSE_Standard,
        Multiplicator_SD_Standard,
        SD_ResponseFunction,
        Mean_Variability_Between,
        SD_Variability_Between)
    
    @rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)
    
        Parameters = quickpsy(Psychometric,Difference,Answer,grouping = .(ID,ConditionOfInterest,StandardValues), bootstrap = "none")$par
        Parameters2 = Parameters %>%
        filter(parn == "p1") %>%
        select(ID,ConditionOfInterest,Mean=par, StandardValues)
        Parameters2$SD = Parameters$par[Parameters$parn == "p2"]
        FittedPsychometricFunctions = Parameters2
    
        FittedPsychometricFunctions$StandardValues = as.character(FittedPsychometricFunctions$StandardValues)
        Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric FittedPsychometricFunctions
    
    Psychometric[:StandardValuesAsFactor] = "placeholder"
    
    formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
    GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=true)
    
    formula2 = @formula(Mean ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelMean = fit(MixedModel,formula2, FittedPsychometricFunctions)
    
    formula3 = @formula(SD ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelSD = fit(MixedModel,formula3, FittedPsychometricFunctions)

    [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4];(coeftable(TwoLevelMean)).cols[4][2];(coeftable(TwoLevelSD)).cols[4][2]]
    
end

SimulateDataframe_Twolevel (generic function with 1 method)

In [ ]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [40,60]
Range_PSE_Difference = [-0.1,-0.05,0,0.05,0.1]
Range_JND_Difference = [-0.2,-0.1,0,0.1,0.2]
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 100
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                TimeStartTrial = Dates.now()
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                Pvalues_Accuracy_TwoLevel = []
                Pvalues_Precision_TwoLevel = []
                
                for j in 1:nIterations
                Pvalues = SimulateDataframe_Twolevel(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    Pvalues_Accuracy_TwoLevel = [Pvalues_Accuracy_TwoLevel;Pvalues[3]]
                    Pvalues_Precision_TwoLevel = [Pvalues_Precision_TwoLevel;Pvalues[4]]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        Duration = ((Dates.now()) - TimeStartTrial))   

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        Duration=((Dates.now()) - TimeStartTrial))
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end
                
                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", PSE_Difference, " ", JND_Difference, " ", mean(Pvalues_Accuracy .< 0.05), " ", 
                    mean(Pvalues_Precision .< 0.05), " ", PowerfulDataframe[!,:Duration][CurrentRunthrough], " END. ")

            end
            CSV.write(join([reps,"_", PSE_Difference, "_", JND_Difference, ".csv"]),PowerfulDataframe)
        end
    end
end



In [ ]:
function SimulateDataframe(n,
        ConditionOfInterest,
        StandardValues,
        reps,
        PSE_Difference,
        JND_Difference,
        Multiplicator_PSE_Standard,
        Multiplicator_SD_Standard,
        SD_ResponseFunction,
        Mean_Variability_Between,
        SD_Variability_Between)
    
    @rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    TimeBeforeDataframeinR = Dates.now()
    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)
    
        Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric FittedPsychometricFunctions

    DurationSimulateDataframe = Dates.now() - TimeBeforeDataframeinR
    
    TimeStartTrial = Dates.now()
    
    formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
    GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=false)
    
    DurationGLMM = ((Dates.now()) - TimeStartTrial)
    [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4];DurationGLMM;DurationSimulateDataframe]
    
end

In [47]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
Range_PSE_Difference = 0.1
Range_JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                DurationsGLMM = []
                DurationsDataframe = []

                for j in 1:nIterations
                Pvalues = SimulateDataframe(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    DurationsGLMM = [DurationsGLMM;Dates.value(Pvalues[3])]
                    DurationsDataframe = [DurationsDataframe;Dates.value(Pvalues[4])]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        Mean_DurationGLMM = mean(DurationsGLMM),
                        SD_DurationGLMM = std(DurationsGLMM),
                        Mean_DurationDataframe = mean(DurationsDataframe),
                        SD_DurationDataframe = std(DurationsDataframe),
                        nIterations = nIterations)

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        Mean_DurationGLMM = mean(DurationsGLMM),
                        SD_DurationGLMM = std(DurationsGLMM),
                        Mean_DurationDataframe = mean(DurationsDataframe),
                        SD_DurationDataframe = std(DurationsDataframe),
                        nIterations = nIterations)
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end

                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", 
                    PowerfulDataframe[!,:Mean_DurationGLMM][CurrentRunthrough], " END. ")

            end
        end
    end
end


5RUNTHROUGH 1 out of 24: 10 30 492.08 END. 5RUNTHROUGH 2 out of 24: 12 30 657.44 END. 5RUNTHROUGH 3 out of 24: 14 30 716.48 END. 5RUNTHROUGH 4 out of 24: 16 30 869.4 END. 5RUNTHROUGH 5 out of 24: 18 30 903.76 END. 5RUNTHROUGH 6 out of 24: 20 30 1113.72 END. 5RUNTHROUGH 7 out of 24: 10 40 725.96 END. 5RUNTHROUGH 8 out of 24: 12 40 826.36 END. 5RUNTHROUGH 9 out of 24: 14 40 1047.92 END. 5RUNTHROUGH 10 out of 24: 16 40 1806.08 END. 5RUNTHROUGH 11 out of 24: 18 40 1933.44 END. 5RUNTHROUGH 12 out of 24: 20 40 2041.36 END. 5RUNTHROUGH 13 out of 24: 10 50 887.52 END. 5RUNTHROUGH 14 out of 24: 12 50 1015.4 END. 5RUNTHROUGH 15 out of 24: 14 50 2000.76 END. 5RUNTHROUGH 16 out of 24: 16 50 2244.52 END. 5RUNTHROUGH 17 out of 24: 18 50 2301.52 END. 5RUNTHROUGH 18 out of 24: 20 50 2525.72 END. 5RUNTHROUGH 19 out of 24: 10 60 1292.6 END. 5RUNTHROUGH 20 out of 24: 12 60 1752.84 END. 5RUNTHROUGH 21 out of 24: 14 60 2307.2 END. 5RUNTHROUGH 22 out of 24: 16 60 2597.84 END. 5RUNTHROUGH 23 out of 24: 18 60

"Durations.csv"

0 milliseconds

In [11]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
reps = 120
PSE_Difference = 0.1
JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
n = 20

20

In [50]:
@rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)

    Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric

,ID,ConditionOfInterest,StandardValues,reps,PSE_Factor_ID,SD_Factor_ID
,Categorical…,Int64,String,Int64,Float64,Float64
1,s1,0,5,1,0.995661,1.10506
2,s2,0,5,1,0.967773,0.955779
3,s4,0,5,1,1.02584,0.819184
4,s5,0,5,1,0.925219,1.1126
5,s6,0,5,1,0.797864,0.915151
6,s7,0,5,1,1.00289,1.13926
7,s8,0,5,1,0.976293,1.13152
8,s9,0,5,1,0.979974,0.953678
9,s10,0,5,1,1.07072,1.01988


In [14]:
 GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Bernoulli()))

ArgumentError: ArgumentError: invalid NLopt arguments: zero step size

In [59]:
formula1 = @formula(Answer ~ Difference*ConditionOfInterest + 
    (Difference + ConditionOfInterest | ID) + 
    (Difference + ConditionOfInterest | StandardValues));
GLMM2 = fit!(GeneralizedLinearMixedModel(formula1, 
        Psychometric, 
        Bernoulli()),
        verbose = true,
        fast = true)

varyβ = true, obj₀ = 9481.26652949665, β = [0.1908301139202959, 1.1842770898509487, -0.8088123900525568, -0.1557891012997059]
   1: 8438.202557895045
   2: 7748.443645276328
   3: 7650.460533890726
   4: 7642.756999268895
   5: 7642.526943735007
   6: 7642.524695788832
   7: 7642.524694611602
f_1: 7642.524694611602 [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 10966.667885372683, β = [0.2598465012231814, 2.6653026602421375, -1.607238439058622, -0.38192830254525006]
   1: Inf
       1, Inf
       2, 11432.175106103876
       3, 8753.99687544543
   2: 7869.211401534454
   3: 7649.148377022627
   4: 7629.169390331179
   5: 7628.177040502164
   6: 7628.1620668404385
   7: 7628.162030679362
   8: 7628.162030679006
f_2: 7628.162030679006 [1.75, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 11033.497853773122, β = [0.27427205570944424, 2.7534860351753, -1.6186875520703838, -0.45494201597463774]
   1: Inf
       1, Inf
       2,

varyβ = true, obj₀ = 10922.071461497206, β = [0.2535044366942136, 2.628580393209925, -1.6075120462893524, -0.33391821873517896]
   1: Inf
       1, Inf
       2, 11345.373180302795
       3, 8733.518605641098
   2: 7869.702720905855
   3: 7655.926133885057
   4: 7638.724104891321
   5: 7637.951540211754
   6: 7637.941299537912
   7: 7637.941280331647
   8: 7637.941280331544
f_20: 7637.941280331544 [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.25, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 10945.82069122794, β = [0.2600199773176801, 2.665335374301473, -1.6072041893654658, -0.3820692720298322]
   1: Inf
       1, Inf
       2, 11441.046330616782
       3, 8740.14809902145
   2: 7874.347072088489
   3: 7660.6492560664
   4: 7643.445722989423
   5: 7642.684807908455
   6: 7642.674829084823
   7: 7642.674810937227
   8: 7642.674810937136
f_21: 7642.674810937136 [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, -1.0, 0.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 10945.683347193522, β = [0.2600659937874791, 2.66529793640

       1, 8115.2552471014715
   4: 7690.871478152258
   5: 7592.251937573044
   6: 7575.60828954595
   7: 7575.130286102038
   8: 7575.125944232268
   9: 7575.125939886672
f_34: 7575.125939886672 [1.4022589130163257, -0.09777142847484219, -0.010264641278527109, 0.4816270429227273, -0.1304226932896262, 0.0, 0.9148704509661871, 0.006022248144083445, -0.0012752256908584603, 1.0603044933673165, -0.0035154980348900723, 0.8893984385175331]
varyβ = true, obj₀ = 10907.301826135248, β = [0.25897264052249325, 2.6500890492025535, -1.626051368854341, -0.3294062340095253]
   1: Inf
       1, 18119.544083082284
       2, 9711.345534374
   2: 10274.62848895675
       1, 8284.317573026934
   3: 9373.642115569366
       1, 7799.832827449278
   4: 7596.86080339324
   5: 7576.817852488844
   6: 7576.007926544866
   7: 7576.00003240886
   8: 7576.000022086431
   9: 7576.000022086404
f_35: 7576.000022086404 [1.362407168956244, -0.07146269304802108, -0.009798816649038975, 0.46127622890444786, -0.09517642838

f_46: 7569.184258529388 [1.4107153410355129, -0.14144814710656645, -0.14451880102755416, 0.3689538737310243, -0.0783669814454554, 0.0, 0.9201252756521484, -0.014493733319075029, -0.002796995273383962, 0.9836330753363952, -0.030281233026610135, 0.8544592850807886]
varyβ = true, obj₀ = 10883.324129104334, β = [0.26406871615537814, 2.721579531386814, -1.5844632476843075, -0.48087654210341707]
   1: Inf
       1, 15746.34834415509
       2, 8940.600624064618
   2: 8279.254808191654
   3: 9702.487557879514
       1, 7754.143545320419
   4: 7584.869416021184
   5: 7570.156114026206
   6: 7569.746719671198
   7: 7569.743951756819
   8: 7569.7439504796175
f_47: 7569.7439504796175 [1.4169173474910197, -0.14658280906368068, -0.15065467111344244, 0.3432131067563211, -0.038365952171069986, 0.0, 0.9047595972018395, -0.08294187315344195, 0.005381243120628718, 1.0164748414199873, -0.03048997899315601, 0.854866922847732]
varyβ = true, obj₀ = 10881.913481923746, β = [0.2619852064606654, 2.7193874466976

f_58: 7562.644394079287 [1.5120712072496123, -0.08232816609290713, -0.08949035495093294, 0.11243121842757331, -0.11391657026969833, 0.0, 0.1641525877524056, -0.07623421098046851, 0.044035861170895925, 0.9269473933656796, -0.014822293273622583, 0.0]
varyβ = true, obj₀ = 10844.235519414853, β = [0.2617015998280871, 2.6664540491503765, -1.5903075941554818, -0.41182634519900746]
   1: Inf
       1, 15442.153902288097
       2, 8830.740078653296
   2: 8051.242908274005
   3: 8258.673135818888
       1, 7630.611561933094
   4: 7568.024330495588
   5: 7564.816333823687
   6: 7564.77337442407
   7: 7564.773253891278
   8: 7564.7732538886185
f_59: 7564.7732538886185 [1.4157306322368506, -0.164647522425187, -0.09117198070296265, 0.32378683743613323, -0.09557883051787283, 0.09999999999999998, 0.7318499889817516, 0.0010566200809599274, 0.012053502799308974, 0.9033238026616728, 0.0036296689882911815, 0.28907671675159397]
varyβ = true, obj₀ = 10862.502502354391, β = [0.2528541287187771, 2.6961597498

...

varyβ = true, obj₀ = 10874.125552646383, β = [0.28944526339099697, 2.7483233793267927, -1.579364443364727, -0.520214015937778]
   1: Inf
       1, 14100.899469477237
       2, 8454.27162437635
   2: 7639.228461586988
   3: 7553.059436118702
   4: 7547.38868625861
   5: 7547.290727241056
   6: 7547.290249807245
   7: 7547.290249758884
f_303: 7547.290249758884 [1.751121067257553, -0.05639044131333651, -0.24894351621737026, 0.2757224071617752, -0.08817360800737574, 2.91479012262355e-5, 0.0, -0.18541833957914944, 0.0031438135566130084, 0.5210196741712864, -0.008826823448142618, 4.273244660648627e-5]
varyβ = true, obj₀ = 10874.12421319221, β = [0.2894461063599822, 2.74829613765646, -1.5793768129821797, -0.5201650940495876]
   1: Inf
       1, 14101.70911889412
       2, 8454.433345796157
   2: 7639.29296101907
   3: 7553.065367553588
   4: 7547.388850909351
   5: 7547.290730315896
   6: 7547.290251716451
   7: 7547.290251667852
f_304: 7547.290251667852 [1.7511153027842086, -0.05638077883183

       1, 14101.070394229384
       2, 8454.254749687188
   2: 7639.218393944309
   3: 7553.0603654788865
   4: 7547.3887222993735
   5: 7547.290722012028
   6: 7547.290244221962
   7: 7547.290244173526
f_317: 7547.290244173526 [1.751094765585829, -0.056439476994287564, -0.24901076885540777, 0.2757124629623731, -0.0882767454265293, 4.3952493470724585e-5, 0.0, -0.18549947666220612, 0.003122363210056026, 0.5212632676362056, -0.008824778148093699, 2.0418456917252562e-5]
varyβ = true, obj₀ = 10874.119428939826, β = [0.2894387403807964, 2.7483417875015843, -1.5793605199461946, -0.5202503773262519]
   1: Inf
       1, 14101.238711787848
       2, 8454.286060740647
   2: 7639.230099450077
   3: 7553.0614791240905
   4: 7547.388755602855
   5: 7547.290723502521
   6: 7547.29024547609
   7: 7547.290245427605
f_318: 7547.290245427605 [1.7509502346470485, -0.056441751481105945, -0.24899195236250493, 0.2757189313740414, -0.08828860020410487, 2.8575476994880507e-5, 0.0, -0.1854765410568675, 0.00312

f_330: 7547.290240084833 [1.751287350579149, -0.05644844902695184, -0.24900846512069674, 0.2757026843198861, -0.08821605694123935, 3.0758066743533246e-5, 0.0, -0.18559403607222036, 0.0031610435786195287, 0.5215101216009786, -0.008850385985614953, 0.0]
varyβ = true, obj₀ = 10874.147951541432, β = [0.28944056872675994, 2.748340594954805, -1.57936890630017, -0.5202288798463758]
   1: Inf
       1, 14100.343948169117
       2, 8454.13161362059
   2: 7639.178636387734
   3: 7553.055772320329
   4: 7547.388582756617
   5: 7547.290716653955
   6: 7547.29023986682
   7: 7547.290239818593
f_331: 7547.290239818593 [1.7513461348475174, -0.056449789672492486, -0.24898463454723865, 0.2756863266297432, -0.08820426762276325, 9.66330606114915e-6, 0.0, -0.1855959806037452, 0.0031597293500396405, 0.5215408130943289, -0.00886456874215806, 0.0]
varyβ = true, obj₀ = 10874.152770716706, β = [0.2894400848722893, 2.748327100275225, -1.5793758653031162, -0.5202006941547443]
   1: Inf
       1, 14100.7405758371

   7: 7547.290238738128
f_344: 7547.290238738128 [1.751279724390669, -0.05649343067797895, -0.24895558593339986, 0.27570671407265923, -0.0882383853068999, 0.0, 0.0, -0.18556778658362097, 0.0031487945154751694, 0.5216211151230363, -0.008846261034884693, 1.3882157995219704e-6]
varyβ = true, obj₀ = 10874.151029161692, β = [0.28942762891576335, 2.7483250195651636, -1.579378536690187, -0.5201974293056986]
   1: Inf
       1, 14100.9942913827
       2, 8454.261033800633
   2: 7639.225577114935
   3: 7553.059991988554
   4: 7547.388703831045
   5: 7547.290716432817
   6: 7547.290238744133
   7: 7547.290238695719
f_345: 7547.290238695719 [1.751295664387596, -0.05649417656872628, -0.24895816396983705, 0.275706771983466, -0.08824031134501305, 0.0, 0.0, -0.18556769806683485, 0.003147691504230206, 0.5216463537569228, -0.00884681152168676, 0.0]
varyβ = true, obj₀ = 10874.153436996236, β = [0.289427654504972, 2.748326176664725, -1.5793789113896781, -0.5201980496069507]
   1: Inf
       1, 14100.8169

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  Answer ~ 1 + Difference + ConditionOfInterest + Difference & ConditionOfInterest + (1 + Difference + ConditionOfInterest | ID) + (1 + Difference + ConditionOfInterest | StandardValues)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 7547.2902

Variance components:
                      Column         Variance     Std.Dev.    Corr.
ID             (Intercept)          2.4971193613 1.580227630
               Difference           0.0644824469 0.253933942 -0.20
               ConditionOfInterest  0.0567952637 0.238317569 -0.94 -0.14
StandardValues (Intercept)          0.0000000000 0.000000000
               Difference           0.2496036171 0.499603460   NaN
               ConditionOfInterest  0.0000718117 0.008474179   NaN -1.00

 Number of obs: 8784; levels of grouping factors: 20, 2

Fixed-effects parameters:
───────────────────────────────────────────────────────────────────────────
               

In [53]:
verbaggform = @formula(r2 ~ (m|id) + (m|item));
gm1a = fit!(GeneralizedLinearMixedModel(verbaggform, 
        dat[:VerbAgg],
        Bernoulli()), 
        verbose = true,
        fast = false)

varyβ = true, obj₀ = 11537.4321647439, β = [-0.09553673453397096]
   1: 8424.125470328892
   2: 8175.430301981829
   3: 8156.265904211067
   4: 8155.651353611125
   5: 8155.648278214871
   6: 8155.648277948747
f_1: 8155.648277948747 [1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 11844.83844993634, β = [-0.06089628432231471]
   1: 8514.668180878241
   2: 8176.001253069019
   3: 8138.994145532149
   4: 8136.989074175192
   5: 8136.962203379701
   6: 8136.962188565084
   7: 8136.962188565075
f_2: 8136.962188565075 [1.75, 0.0, 1.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 11664.00273650668, β = [-0.052070779955305624]
   1: 8603.072090222
   2: 8264.207958870173
   3: 8219.01434959314
   4: 8216.246613253423
   5: 8216.206416563964
   6: 8216.206391322778
   7: 8216.206391322748
f_3: 8216.206391322748 [1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 11801.03848743459, β = [-0.07839938514016795]
   1: 8563.993202132338
   2: 8243.739833434034
   3: 8204.823112447244
   4: 8202.5063753555

   5: 8118.206834037523
   6: 8118.206833359988
f_27: 8118.206833359988 [1.4263828375877787, -0.14999374003097815, 1.0126564745353694, 1.1062777033694022, -0.41647333647660273, 1.0778863328881196]
varyβ = true, obj₀ = 11737.59382194211, β = [-0.11939078464599105]
   1: 8450.440298879446
   2: 8147.092848393851
   3: 8119.250068634069
   4: 8118.137986440358
   5: 8118.130705911973
   6: 8118.1307050709265
f_28: 8118.1307050709265 [1.4438251002810139, -0.15348064568604142, 1.0385986195331278, 1.0992238268786267, -0.3276158711274158, 1.0901286203841567]
varyβ = true, obj₀ = 11752.97593718699, β = [-0.10335056719202707]
   1: 8457.472830164787
   2: 8148.876230882531
   3: 8120.023305752707
   4: 8118.848087651869
   5: 8118.840233040019
   6: 8118.840232088568
f_29: 8118.840232088568 [1.4269147688416433, -0.1505579175761898, 1.0928013967701544, 1.1278277746357874, -0.28912444786418623, 1.062839161785352]
varyβ = true, obj₀ = 11738.507511783111, β = [-0.10905352474863772]
   1: 8452.08343

   2: 8146.748503233587
   3: 8118.910362683137
   4: 8117.794474441589
   5: 8117.787025931521
   6: 8117.787025033422
f_51: 8117.787025033422 [1.4326421617574732, -0.12205362112039941, 1.0146899672029583, 1.0273981248633186, -0.32157642502655426, 1.1844124489782673]
varyβ = true, obj₀ = 11725.43050816685, β = [-0.07051898261309597]
   1: 8450.368712624482
   2: 8146.815297210567
   3: 8118.920939935166
   4: 8117.798725184261
   5: 8117.791153127116
   6: 8117.791152195739
f_52: 8117.791152195739 [1.436524837259671, -0.11753259705008591, 1.0079861488769923, 1.0111357755607255, -0.30208754530870974, 1.203130771408462]
varyβ = true, obj₀ = 11723.038403628443, β = [-0.061234240952403055]
   1: 8448.839176507134
   2: 8146.445640849248
   3: 8118.876838062272
   4: 8117.7828719011495
   5: 8117.77571625899
   6: 8117.7757154224255
f_53: 8117.7757154224255 [1.4191420495266462, -0.12500918822560486, 1.0174725462830674, 1.0256310505846835, -0.32630710603910557, 1.188204498826188]
varyβ = tr

f_75: 8117.707193062496 [1.4225795978197728, -0.12079430523622134, 1.0193361419667295, 0.9691502973312868, -0.2846411414347507, 1.2347413494905704]
varyβ = true, obj₀ = 11727.907532067218, β = [-0.0449910002483549]
   1: 8450.31616960362
   2: 8146.664887823077
   3: 8118.829018567289
   4: 8117.714653858872
   5: 8117.707222230049
   6: 8117.7072213283445
f_76: 8117.7072213283445 [1.4231932891256351, -0.12113065029667047, 1.0197280130009534, 0.969905900726263, -0.2838811941500491, 1.234720348004867]
varyβ = true, obj₀ = 11727.493881699218, β = [-0.04507185654415512]
   1: 8450.12345275044
   2: 8146.625354588958
   3: 8118.825569214912
   4: 8117.713816363279
   5: 8117.706415484986
   6: 8117.706414589882
f_77: 8117.706414589882 [1.4231072455223108, -0.12217936983411196, 1.0190629316065032, 0.9693446801984186, -0.28373919229766564, 1.2343320427476048]
varyβ = true, obj₀ = 11727.394367762738, β = [-0.04498545252878663]
   1: 8450.011738079205
   2: 8146.5983222634795
   3: 8118.823049

   6: 8117.703847214434
f_98: 8117.703847214434 [1.4248780094856377, -0.12509068799669643, 1.0180026882621243, 0.9693739579319673, -0.3019611063198081, 1.2291756437805155]
varyβ = true, obj₀ = 11726.804061897054, β = [-0.04779418570859973]
   1: 8449.843260008662
   2: 8146.563978754355
   3: 8118.818850974308
   4: 8117.711203433346
   5: 8117.703858460553
   6: 8117.703857580467
f_99: 8117.703857580467 [1.4248747129403116, -0.12492384209757453, 1.0180119075447072, 0.969203669352264, -0.30231798453135617, 1.2290179054382884]
varyβ = true, obj₀ = 11726.779615907779, β = [-0.04782774504469326]
   1: 8449.823912095342
   2: 8146.559471701655
   3: 8118.818499673229
   4: 8117.71118231446
   5: 8117.7038414808885
   6: 8117.703840601615
f_100: 8117.703840601615 [1.4248041593109801, -0.1251616601424228, 1.0180426864076375, 0.9695013594057501, -0.30227596310053173, 1.2293373575977198]
varyβ = true, obj₀ = 11726.77859426012, β = [-0.047823842939792284]
   1: 8449.82152465385
   2: 8146.55886

f_121: 8117.703824111219 [1.4248290795943621, -0.1252024768776387, 1.018402178050455, 0.9696047770194933, -0.30237210237058265, 1.2293380422275606]
varyβ = true, obj₀ = 11726.92583127275, β = [-0.047862425652792466]
   1: 8449.871835332218
   2: 8146.567805327317
   3: 8118.818993138106
   4: 8117.711170721324
   5: 8117.70382497372
   6: 8117.703824093508
f_122: 8117.703824093508 [1.4248330564999412, -0.1252051911539735, 1.018399078339009, 0.9695796042632286, -0.30233080895093717, 1.2293615443994297]
varyβ = true, obj₀ = 11726.929586894943, β = [-0.047848098800483706]
   1: 8449.872577880646
   2: 8146.567912225674
   3: 8118.818999921893
   4: 8117.711170805775
   5: 8117.703824975262
   6: 8117.703824095032
f_123: 8117.703824095032 [1.4248352132881406, -0.12520594774736168, 1.0183996928724515, 0.9695727605777168, -0.3023362042492962, 1.229376097815144]
varyβ = true, obj₀ = 11726.925207831357, β = [-0.04784438659914994]
   1: 8449.871177162948
   2: 8146.5676727826485
   3: 8118.8189

   4: 8117.711207691849
   5: 8117.703848052738
   6: 8117.703847170782
f_20: 8117.703847170782 [-0.05149609645305187, 1.4257835698587167, -0.12506466141495268, 1.018939286415677, 0.9666526357285878, -0.30107148963877606, 1.2301970573183008]
varyβ = false, obj₀ = 11728.111558663612
   1: 8450.448458257824
   2: 8146.677993576473
   3: 8118.829151067292
   4: 8117.714216518982
   5: 8117.706789019978
   6: 8117.706788124644
f_21: 8117.706788124644 [-0.0513827350267183, 1.428977758181214, -0.1250598815859572, 1.0189723382061215, 0.9706578662938341, -0.3070509805218045, 1.2272768003165828]
varyβ = false, obj₀ = 11726.375326240839
   1: 8449.84733356655
   2: 8146.568695460532
   3: 8118.818718536116
   4: 8117.7109254426905
   5: 8117.703582208337
   6: 8117.703581328972
f_22: 8117.703581328972 [-0.05200147726309854, 1.4247862583958264, -0.12507652863007654, 1.0188065599525542, 0.9696783462672147, -0.2952942644030685, 1.2295527039937653]
varyβ = false, obj₀ = 11726.390359458503
   1: 8449

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  r2 ~ 1 + (1 + m | id) + (1 + m | item)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 8117.7034

Variance components:
        Column    Variance   Std.Dev.    Corr.
id   (Intercept)  1.76802958 1.32967274
     m: do        0.91685601 0.95752598 -0.12
item (Intercept)  0.81884174 0.90489875
     m: do        1.39470367 1.18097573 -0.24

 Number of obs: 7584; levels of grouping factors: 316, 24

Fixed-effects parameters:
──────────────────────────────────────────────────────
               Estimate  Std.Error    z value  P(>|z|)
──────────────────────────────────────────────────────
(Intercept)  -0.0528834     0.2291  -0.230832   0.8174
──────────────────────────────────────────────────────

In [47]:
dat[:VerbAgg]

,a,g,item,resp,id,b,s
,Int32,Categorical…,Categorical…,Categorical…,Categorical…,Categorical…,Categorical…
1,20,M,S1WantCurse,no,1,curse,other
2,11,M,S1WantCurse,no,2,curse,other
3,17,F,S1WantCurse,perhaps,3,curse,other
4,21,F,S1WantCurse,perhaps,4,curse,other
5,17,F,S1WantCurse,perhaps,5,curse,other
6,21,F,S1WantCurse,yes,6,curse,other
7,39,F,S1WantCurse,yes,7,curse,other
8,21,F,S1WantCurse,no,8,curse,other
9,24,F,S1WantCurse,no,9,curse,other


In [34]:
verbaggform = @formula(r2 ~ (m|id)+(m|item));
gm1a = fit!(GeneralizedLinearMixedModel(verbaggform, dat[:VerbAgg],
    Bernoulli()), fast=false)

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  r2 ~ 1 + a + g + b + s + (1 + m | id) + (1 + m | item)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 8064.2288

Variance components:
        Column     Variance   Std.Dev.   Corr.
id   (Intercept)  1.801874970 1.3423394
     m: do        0.896334055 0.9467492 -0.23
item (Intercept)  0.041683269 0.2041648
     m: do        0.639784817 0.7998655 -0.09

 Number of obs: 7584; levels of grouping factors: 316, 24

Fixed-effects parameters:
──────────────────────────────────────────────────────
               Estimate  Std.Error    z value  P(>|z|)
──────────────────────────────────────────────────────
(Intercept)   0.459534   0.363961     1.26259   0.2067
a             0.0531583  0.0163353    3.2542    0.0011
g: M          0.243634   0.186424     1.30688   0.1913
b: scold     -1.05134    0.163385    -6.43477   <1e-9
b: shout     -1.76674    0.164763   -10.7229    <1e-26
s: self      -1.028      0.1335

In [41]:
using RData
datf = joinpath(dirname(pathof(MixedModels)),"..","test","dat.rda")
const dat = Dict(Symbol(k)=>v for (k,v) in load(datf))

7584-element CategoricalArray{String,1,UInt8}:
 "N"
 "N"
 "Y"
 "Y"
 "Y"
 "Y"
 "Y"
 "N"
 "N"
 "Y"
 "Y"
 "Y"
 "Y"
 ⋮
 "N"
 "Y"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"